In [ ]:
import numpy as np
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import os

In [ ]:
# check if gpu/cpu
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# Define the paths of download and result

ROOT = './'

DataPath = ROOT + 'Classifier-data/'
ResultPath = ROOT + 'Classifier-results/'

# make these directories if not available 
# --> this can help avoid to download again and again
os.makedirs(DataPath, exist_ok=True)
os.makedirs(ResultPath, exist_ok=True)

In [ ]:
# Load the MNIST datasets
train_set = torchvision.datasets.MNIST(
    root=DataPath,
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_set = torchvision.datasets.MNIST(
    root=DataPath,
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [ ]:
# check data 
data, label = train_set[4]
print(data.size())
print(label)

In [ ]:
# learn parameters of torch.utils.data.DataLoader(
   # dataset train_set,
   # amount of lines of each inputing time    batch_size=16,
   # when each iteration, whether data need to be shuffle/disrupt   shuffle=True,
   # amount of works, the best value is the amount of cpu/gpu of laptop num_workers=2
#)

# met mistakes about num_workers, in Jupyter num_worker = 0 is more friendly


train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=16,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=24, # Forward pass only so batch size can be larger
    shuffle=False,
    num_workers=0
)

classes = np.arange(0, 10)
print(classes)

In [ ]:
# Show some images and labels as a sanity check.
# Use `torchvision.utils.make_grid` to create one image from a set of images. 
# Note that this function converts single channel (grey-scale) tensors to have three channels. 
# This is done by replicating the values into red, green and blue channels.


def timshow(x):
    # np.transpose can change axises order by order of the instruction(parameters)
    xa = np.transpose(x.numpy(),(1,2,0))  
    plt.imshow(xa)
    plt.axis('off')
    plt.show()
    return xa
    
# get a batch of random training examples (images and corresponding labels)
dataiter = iter(train_loader)
# images, labels = dataiter.next()
images, labels = next(dataiter)

# show images and labels
print(images.size())
timshow(torchvision.utils.make_grid(images))
print(*labels.numpy())     # asterisk unpacks the ndarray


## Build a classifier
Instead of defining the classifier function, loss function and parameter updates directly as we did in PyTorch.ipynb, it is convenient to use the `torch.nn` and `torch.optim` packages. These provide a simple way to build networks without losing sight of the iterative steps in gradient descent.

First we construct the classifer function using the nn.Sequential wrapper that simply sequences the steps in the classifier function. In the case of a linear classifier there is just one nn.Linear layer. This is preceeded by `nn.Flatten` that vectorises a $28\times28$ input image into a 1D vector of length $28*28$. We will also experiment with a two layer classifier.

In [ ]:
input = torch.randn(32, 1, 5, 5)
# With default parameters
m = nn.Flatten()
output = m(input)
output.size()
#torch.Size([32, 25])
# With non-default parameters
m = nn.Flatten(0, 2)
output = m(input)
output.size()
#torch.Size([160, 5])

In [ ]:
# one layer net

net1 = nn.Sequential(
    nn.Flatten(),
    
    # single layer
    nn.Linear(28*28, 10)
    

)

#  .parameters()得到所有参数，第一次循环输出的是权重weight，与输入保持一致
#  .parameters() gets all parameters, first loop figure out the weight(whose form is the same with inputs)
# 第二次循环输出的是偏执bias数量，与输出数量保持一致
# the second loop come out bias, whose form is the same with outputs
for param in net1.parameters():
    print(param.shape)
    print(param)


In [ ]:
# two layers net
# Sigmod between layers

net2 = nn.Sequential(
    nn.Flatten(),

    # single layer
    #nn.Linear(28*28, 10)
    
    # two layers
    nn.Linear(28*28, 300),
    nn.Sigmoid(),
    nn.Linear(300,10)
)

for param in net2.parameters():
    print(param.shape)

In [ ]:
nepochs = 100    # number of epochs --> you can change as you want, for e.g., try 200 epochs
net = net1       # 1-layer model
results_path = ResultPath + 'linear1layer200epochs.pt'

# initialise ndarray to store the mean loss in each epoch (on the training data)
losses = np.zeros(nepochs)

# Use a loss function and optimiser provided as part of PyTorch.
# The chosen optimiser (Stochastic Gradient Descent with momentum) needs only to be given the parameters (weights and biases)
# of the network and updates these when asked to perform an optimisation step below.
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(nepochs):  # loop over the dataset multiple times

    # initialise variables for mean loss calculation
    running_loss = 0.0
    n = 0
    
    for data in train_loader:
        inputs, labels = data
        
        # Zero the parameter gradients to remove accumulated gradient from a previous iteration.
        # 将所有参数梯度清零
        optimizer.zero_grad()

        # Forward, backward, and update parameters
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        # 运算后将梯度加入到grid属性中
        # after calling, add all gradients into grids
        loss.backward()
        # 通过梯度下降优化参数，梯度由backward产生
        # Gradients create by backward(), optimize parameters by gradient down
        optimizer.step()
    
        # accumulate loss and increment minibatches
        running_loss += loss.item()
        n += 1
       
    # record the mean loss for this epoch and show progress
    losses[epoch] = running_loss / n
    print(f"epoch: {epoch+1} loss: {losses[epoch] : .3f}")
    
# save network parameters and losses
torch.save({"state_dict": net.state_dict(), "losses": losses}, results_path)
   

In [ ]:
nepochs = 100   # number of epochs --> you can change as you want
net = net2       # 2-layer model
results_path = ResultPath + '/linear2layer200epochs.pt'

# initialise ndarray to store the mean loss in each epoch (on the training data)
losses = np.zeros(nepochs)

# Use a loss function and optimiser provided as part of PyTorch.
# The chosen optimiser (Stochastic Gradient Descent with momentum) needs only to be given the parameters (weights and biases)
# of the network and updates these when asked to perform an optimisation step below.
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(nepochs):  # loop over the dataset multiple times

    # initialise variables for mean loss calculation
    running_loss = 0.0
    n = 0
    
    for data in train_loader:
        inputs, labels = data
        
        # Zero the parameter gradients to remove accumulated gradient from a previous iteration.
        optimizer.zero_grad()

        # Forward, backward, and update parameters
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
    
        # accumulate loss and increment minibatches
        running_loss += loss.item()
        n += 1
       
    # record the mean loss for this epoch and show progress
    losses[epoch] = running_loss / n
    print(f"epoch: {epoch+1} loss: {losses[epoch] : .3f}")
    
# save network parameters and losses
torch.save({"state_dict": net.state_dict(), "losses": losses}, results_path)

In [ ]:
# Notice that the first dimension of inputs and outputs corresponds to a minibatch of examples.
print(f"input size: {inputs.size()}, output size: {outputs.size()}")

In [ ]:
# Compare the history of the loss function during training (mean loss in each epoch) for 1 and 2 layer models
d1 = torch.load(ResultPath +  'linear1layer200epochs.pt')
d2 = torch.load(ResultPath + 'linear2layer200epochs.pt')

fig = plt.figure()

plt.plot(d1["losses"], 'r', label = '1 layer', )
plt.plot(d2["losses"], 'g', label = '2 layers' )
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training loss for 1 and 2 layer classifiers')

fig.savefig(ResultPath + "training_loss_MNIST.svg")
plt.show()

In [ ]:
# trace() 主对角线的和  Sum of the main diagonal elements
# np.diag() 对角线元素 The main diagonal elements

def accuracy(cnfm):
    return cnfm.trace()/cnfm.sum((0,1))

def recalls(cnfm):
    return np.diag(cnfm)/cnfm.sum(1)

def precisions(cnfm):
    return np.diag(cnfm)/cnfm.sum(0)    

In [ ]:
d = torch.load(ResultPath + 'linear2layer200epochs.pt')
# now load the parameter state into the current model (make sure this is the right model).
net.load_state_dict(d["state_dict"])

# initialise confusion matrix
nclasses = classes.shape[0]
cnfm = np.zeros((nclasses,nclasses),dtype=int)

# work without gradient computation since we are testing (i.e. no optimisation)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        
        # find the class with the highest output.
        # note that the outputs are confidence values since we didn't need to apply softmax in our network
        # (nn.crossentropyloss takes raw condifence values and does its own softmax)   
        _, predicted = torch.max(outputs, 1)    
        #outputs返回的是10个类别的概率，选择概率最大的那个
       ## 此处表示返回一个元组中有两个值，但是对第一个不感兴趣
        #返回的元组的第一个元素是image data，即是最大的值；
        #第二个元素是label，即是最大的值对应的索引。由于我们只需要label（最大值的索引），
        #所以有 _ , predicted这样的赋值语句，表示忽略第一个返回值，把它赋值给 _，即舍弃它。
         
        #第二个参数 1 
        #相当于取消掉列这个维度，在每行寻找最大值。
        #axis = 0 表示纵轴， axis = 1 代表横轴
            
        # accumulate into confusion matrix
        for i in range(labels.size(0)):
            cnfm[labels[i].item(),predicted[i].item()] += 1
              
print("Confusion matrix")
print(cnfm)

# show confusion matrix as a grey-level image
plt.imshow(cnfm, cmap='gray')

# show per-class recall and precision
print(f"Accuracy: {accuracy(cnfm) :.1%}")
r = recalls(cnfm)
p = precisions(cnfm)
for i in range(nclasses):
    print(f"Class {classes[i]} : Precision {p[i] :.1%}  Recall {r[i] :.1%}")


## Defining a bespoke model class

In the above, we have used the 'container' module `nn.Sequential` to define our network. To give more flexibility in the definition of the network, we can replace this with our own `nn.module` as below. Notice here, we have used this flexibility to perform the flattening ourselves instead of using `nn.Flatten` - this will be more efficient.

In [ ]:
class Classifier(nn.Module):
    
    def __init__(self, num_classes=10):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(28 * 28, num_classes)
        
    def forward(self, x):
        out = x.reshape(x.size(0), -1) # flatten the input
        out = self.fc1(out)
        return out
    
net = Classifier()
print(net)

In [ ]:
# three layers net
# ReLU between layers
net3 = nn.Sequential(
    nn.Flatten(),

    # single layer
    #nn.Linear(28*28, 10)
    
    # three layers
    nn.Linear(28*28, 300),
    nn.ReLU(),
    nn.Linear(300,50),
    nn.ReLU(),
    nn.Linear(50,10)
    
)

for param in net3.parameters():
    print(param.shape)

In [ ]:
nepochs = 100
net = net3
results_path = ResultPath + '/linear3layer400epochs.pt'
losses = np.zeros(nepochs)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(nepochs):
    running_loss = 0.0
    n=0
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        n += 1
    losses[epoch] = running_loss / n
    print(f"epoch: {epoch+1} loss: {losses[epoch] : .3f}")
torch.save({"state_dict": net.state_dict(), "losses": losses}, results_path)

In [ ]:
d = torch.load(ResultPath + '/linear3layer400epochs.pt')
# now load the parameter state into the current model (make sure this is the right model).
net.load_state_dict(d["state_dict"])

# initialise confusion matrix
nclasses = classes.shape[0]
cnfm = np.zeros((nclasses,nclasses),dtype=int)

# work without gradient computation since we are testing (i.e. no optimisation)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        
        # find the class with the highest output.
        # note that the outputs are confidence values since we didn't need to apply softmax in our network
        # (nn.crossentropyloss takes raw condifence values and does its own softmax)   
        _, predicted = torch.max(outputs, 1)    
        #outputs返回的是10个类别的概率，选择概率最大的那个
       ## 此处表示返回一个元组中有两个值，但是对第一个不感兴趣
        #返回的元组的第一个元素是image data，即是最大的值；
        #第二个元素是label，即是最大的值对应的索引。由于我们只需要label（最大值的索引），
        #所以有 _ , predicted这样的赋值语句，表示忽略第一个返回值，把它赋值给 _，即舍弃它。
         
        #第二个参数 1 
        #相当于取消掉列这个维度，在每行寻找最大值。
        #axis = 0 表示纵轴， axis = 1 代表横轴
            
        # accumulate into confusion matrix
        for i in range(labels.size(0)):
            cnfm[labels[i].item(),predicted[i].item()] += 1
              
print("Confusion matrix")
print(cnfm)

# show confusion matrix as a grey-level image
plt.imshow(cnfm, cmap='gray')

# show per-class recall and precision
print(f"Accuracy: {accuracy(cnfm) :.1%}")
r = recalls(cnfm)
p = precisions(cnfm)
for i in range(nclasses):
    print(f"Class {classes[i]} : Precision {p[i] :.1%}  Recall {r[i] :.1%}")
